# Задание 2. Итерационные методы решения линейных систем

In [4]:
from latexifier import latexify
from IPython.display import display, Markdown
import numpy as np
from typing import Callable, List, Tuple, Union, Any
import warnings
warnings.filterwarnings('ignore')

In [5]:
class Matrix(np.ndarray):
    """extends np.ndarray"""

    def __new__(cls, input_array):
        obj = np.asarray(input_array).view(cls)
        return obj

    def inverse_rows(self, i: int, j: int):
        """Swaps i-th and j-th rows of a matrix"""
        self[i], self[j] = self[j].copy(), self[i].copy()

    def inverse_cols(self, i: int, j: int):
        """Swaps i-th and j-th columns of a matrix"""
        self = self.transpose()
        self.inverse_rows(i, j)
        self = self.transpose()
    
    def tex(self) -> str:
        """Latexifies the matrix rounding up values up to 6 decimal values"""
        return latexify(self)

    def norm(self) -> float:
        """Calculates Matrix norm using np.linalg.norm"""
        return np.linalg.norm(self, 1)

    def inv(self) -> Any:
        return Matrix(np.linalg.inv(self))

In [6]:
def Gauss(A: Matrix, B: Matrix, n: int) -> Matrix:
    """
    Вычисляет решения линейного уравнения Ax = B методом Гаусса с выбором
    главного элемента по столбцам (с перестановкой строк)
    """
    AB = np.append(A, B)
    X = [[0] for _ in range(n)]
    for k in range(n):
        j = k
        for i in range(k+1, n):
            if abs(AB[j][k]) < abs(AB[i][k]):
                j = i
        AB.inverse_rows(k, j)
        for j in range(k+1, n): 
            c = AB[j][k] / AB[k][k]
            for i in range(n+1):
                AB[j][i] -= c * AB[k][i]
    X[n-1][0] = AB[n-1][n] / AB[n-1][n-1]
    for k in range(n-1, -1, -1):
        s = 0
        for i in range(k+1, n):
            s += AB[k][i] * X[i][0]
        X[k][0] = (AB[k][n] - s) / AB[k][k]
    return Matrix(np.array(X))

In [7]:
def append(M: np.ndarray, N: np.ndarray) -> Matrix:
    """Appends two numpy arrays with axis=1"""
    return Matrix(np.append(arr=np.asarray(M), values=N, axis=1))


def print_tex(*argv) -> None:
    """Displays a LaTex Markdown output"""
    res = ' '.join(['$$'] + [(latexify(arg) if isinstance(arg, np.ndarray) else str(arg)) for arg in argv] + ['$$'])
    display(Markdown(res))


def get_latex_column(arg: str, n: int) -> str:
    """
    Prepares a string with LaTex syntax for a column of `arg` with indeces form 1 to `n`
    """
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = arg + "_{" + str(i+1) + "}" + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)


def get_exact_column(col: Matrix, n: int) -> str:
    """Prepares a string with LaTex syntax for a column of {col} elements"""
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = str(float(col[i][0])) + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)


def print_system_output(A: Matrix, s: str, B: Matrix, X: Matrix, n: int) -> None:
    """Oupputs the result of solving a linear equation"""
    B_ = np.matmul(A, X)
    print_tex('X^* = ', get_latex_column("x^*", n), '=', X.tex())
    print_tex(rf'{s} \times X^* = ', A.tex(), X.tex(), '=', B_.tex(), '= B^*')
    print_tex('B = ', get_exact_column(B, n), r'\stackrel{?}{=}', get_exact_column(B_, n), '= B^*')
    print_tex(r'B - B^* = ', get_exact_column(B - B_, n))

In [22]:
n = 4
A = Matrix(np.random.rand(n, n) * 10)
B = Matrix(np.random.rand(n, 1) * 10)
print_tex('A=', A, ',~~', 'B=', B)

$$ A= \begin{pmatrix} 9.933538 & 5.761063 & 6.774131 & 1.413183 \\ 3.053858 & 5.727066 & 1.880276 & 0.192694 \\ 9.480117 & 2.335551 & 5.931607 & 5.934389 \\ 3.902798 & 2.847632 & 3.360107 & 5.975695 \end{pmatrix} ,~~ B= \begin{pmatrix} 1.383478 \\ 2.978910 \\ 3.384812 \\ 3.239904 \end{pmatrix} $$

In [30]:
from cmath import inf


D, D_inv, L, R = [Matrix(np.zeros((n,n))) for _ in range(4)]
for i in range(n):
    D[i][i] = A[i][i]
    D_inv[i][i] = 1 / A[i][i]
    for j in range(n):
        if j < i:
            L[i][j] = A[i][j]
        elif j > i:
            R[i][j] = A[i][j]
H = Matrix(np.identity(n) - D_inv @ A)
G = D_inv @ B
print_tex('H=', H, ',~~', 'G=', G)
print_tex(r'||H||_\infty = ', np.linalg.norm(H, inf))

$$ H= \begin{pmatrix} 0 & -0.579961 & -0.681945 & -0.142264 \\ -0.533233 & 0 & -0.328314 & -0.033646 \\ -1.598238 & -0.393747 & 0 & -1.000469 \\ -0.653112 & -0.476536 & -0.562295 & 0 \end{pmatrix} ,~~ G= \begin{pmatrix} 0.139273 \\ 0.520146 \\ 0.570640 \\ 0.542180 \end{pmatrix} $$

$$ ||H||_\infty =  2.9924534088632235 $$